# **Análise e predição do posicionamento político do eleitor brasileiro**

Projeto final realizado para a disciplina de Ciência de Dados do curso de Ciência da Computação, *Universidade de São Paulo*.

## **Introdução**

O presente trabalho tem como objetivo realizar, primeiramente, uma *análise exploratória* dos dados provenientes de uma pesquisa de opinião sobre o perfil ideológico dos eleitores brasileiros, feita pelo Instituto DataSenado no ano de 2024. O conjunto de dados original contém diversas variáveis de natureza sociodemográfica, comportamental e política, mas nem todas serão utilizadas. Serão mantidas somente aquelas que possam contribuir para compreender a variável alvo (*target*) **Posicionamento Político**, que representa o espectro político autodeclarado por cada participante. A análise de dados buscará, portanto, investigar as relações entre a variável target e as demais variáveis selecionadas, por meio de visualizações gráficas e métodos estatísticos. 

Após a fase exploratória, o projeto avançará para a segunda etapa: a construção de um modelo preditivo. Com o uso de técnicas de *machine learning*, o objetivo será desenvolver um modelo capaz de prever o espectro político com base nas respostas às variáveis selecionadas da pesquisa.

Importante mencionar que este trabalho não se preocupará em fazer inferência sobre a preferência política da população brasileira como um todo, tendo em vista que uma parcela significativa dos dados originais não será utilizada.

* <small>Os dados utilizados neste trabalho foram obtidos por meio dos microdados disponibilizados pelo Instituto DataSenado em <a href="https://www12.senado.leg.br/institucional/datasenado/publicacaodatasenado?id=pesquisa-traca-perfil-ideologico-dos-eleitores-brasileiros" target="_blank" rel="noopener noreferrer">Pesquisa traça perfil ideológico dos eleitores brasileiros</a>.</small>
* <small>Junto com os dados foi disponibilizado um dicionário para auxiliar na interpretação.</small>

## **1. Importação de bibliotecas e configurações gerais**

In [177]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

## **2. Carregamento dos dados**

In [178]:
df = pd.read_csv(r'data\raw\panorama_politico_06_2024\DATASEN BR 2024 06 BAROMETRO - DADOS.csv', sep=';')

## **3. Visão geral dos dados não tratados**

In [179]:
df.shape

(21808, 109)

In [180]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21808 entries, 0 to 21807
Columns: 109 entries, ID to W2
dtypes: float64(49), int64(52), object(8)
memory usage: 18.1+ MB


In [181]:
df.head()

,ID,P04,P06,P07,P08,P09,P10,P11,P12,P13,P14,P15,P16,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37,P38,P39,P40,P41,P42,P43,P44,P45,P46,P47,P48,P49,P50,P51,P52,P53,P54,P55,P56,P57,P58,P59,P60,P61,P62,P63,P64,V02,V03,V05,V06,V07,V08,V09,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,VD_DENGUE_1,VD_DENGUE_2,VD_ACOMPANHA_SENADO,VD_REDE_SOCIAL,VD_TIPO_OCUPACAO_1,VD_TIPO_OCUPACAO_2,VD_BUSCA_TRABALHO,VD_REGIAO,VD_RESIDENCIA,VD_IDADE,VD_RACA,VD_PORTE,VD_TRABALHO,VD_EDUCACAO,POP_UF_SEXO,POP_UF_RACA,POP_UF_IDADE,POP_UF_RESIDENCIA,POP_UF_PORTE,POP_UF_TRABALHO,POP_UF_EDU,W1,W2
0,1,41,3,3,4,2,2,2,2,1,1,2,NaN,NaN,1,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,2,2,2,2,2,1,2,3,1,2,2,1,4,1.0,1,1,2,1,3.0,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,4,4,2,2,2,1,3,3,3,1,1,3,4.0,NaN,4,1,3,1,2,1,4,"4599548,01319187","5928889,83043946","1736626,25524565","8334552,9943779",4791536,"5981782,45942209","2337434,74097726","2,79749300778953","12549,6687231227"
1,2,29,2,2,2,2,2,2,2,2,2,2,NaN,NaN,2,1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1,2,2,2,97,2,1,3,3,4,1,2,3,1.0,1,5,3,2,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1.0,1.0,NaN,5,4,2,1,1,1,97,3,3,2,1,4,NaN,1.0,2,1,2,2,3,2,3,"6203186,58563845","9446410,89141747","2251115,21205504","8765141,45702551",3033950,"970710,50572401","3842125,61215856","1,12345379164713","2805,47489557209"
2,3,28,1,2,2,2,2,2,2,2,2,2,NaN,NaN,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1,2,1,1,1,2,1,2,2,4,1,2,3,1.0,2,1,1,1,3.0,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,4,3,2,1,2,1,1,3,3,4,1,3,5.0,NaN,2,2,1,2,1,1,3,"966058,41748564","1406439,72866629","527384,65232656","371734,75555842",998303,"995087,55192826","527747,57235541","0,335079450803203","3533,35524432939"
3,4,28,3,2,2,2,1,2,2,2,2,1,2.0,1.0,2,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,2,1,1,2,2,2,3,1,1,3,2,2.0,1,1,3,2,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2.0,1.0,NaN,3,5,1,1,1,1,1,2,1,1,2,4,NaN,1.0,2,1,2,2,2,3,1,"966058,41748564","1406439,72866629","374974,77422564","1462103,48138951",608944,"728396,78367523","679620,91966034","0,335079450803203","16466,2467766673"
4,5,24,1,2,1,3,2,2,2,1,2,2,NaN,NaN,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,1.0,1.0,3.0,2.0,1.0,1.0,1.0,1,2,1,1,1,1,1,1,2,1,2,1,3,1.0,1,5,3,1,3.0,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,4,3,2,2,2,4,3,3,3,4,1,3,4.0,NaN,2,1,2,1,2,1,3,"1345034,43392631","1067871,54843917","582338,13699172","2344797,51342096",973772,"1378587,88363752","865902,52059461","0,242935105861798","2386,70789580883"


O conjunto de dados inicialmente contém 21.808 linhas e 109 colunas (perguntas). Cada linha é a resposta de um entrevistado para as 109 perguntas. É possível observar que tanto as perguntas quanto as respostas estão em formato numérico. Dessa forma, para facilitar o entendimento e a análise dos dados, cabe converter esses valores para a forma textual.

## **4. Tratamento dos dados**

### **4.1. Filtrando por colunas importantes**

Após análise detalhada do dicionário do conjunto de dados, foi possível selecionar as perguntas mais relevantes para serem correlacionadas com o espectro político (pergunta **P64**). Nestre trabalho, serão incluídas apenas as respostas que expressem uma **posição política clara (esquerda, direita ou centro)**, de tal forma que serão desconsideradas opções neutras ou indefinidas.

In [182]:
selected_columns = [ 
    "P04",
    "P50",
    "P51",
    "P52",
    "P54",
    "P55",
    "P56",
    "P57",
    "P58",
    "P59",
    "P60",
    "P61",
    "P62",
    "P63",
    "V02",
    "V03",
    "V08",
    "V27",
    "VD_EDUCACAO",
    "VD_REGIAO",
    "VD_IDADE",
    "P64"  # Variável alvo que identifica o espectro político
]

df = df[selected_columns]
df = df[df['P64'].isin([1, 2, 3])].reset_index(drop=True) # Para filtrar apenas respostas: esquerda, direita, centro

In [183]:
df.shape

(12345, 22)

In [184]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12345 entries, 0 to 12344
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   P04          12345 non-null  int64  
 1   P50          8901 non-null   float64
 2   P51          8901 non-null   float64
 3   P52          8901 non-null   float64
 4   P54          8901 non-null   float64
 5   P55          12345 non-null  int64  
 6   P56          12345 non-null  int64  
 7   P57          12345 non-null  int64  
 8   P58          12345 non-null  int64  
 9   P59          12345 non-null  int64  
 10  P60          12345 non-null  int64  
 11  P61          12345 non-null  int64  
 12  P62          12345 non-null  int64  
 13  P63          12345 non-null  int64  
 14  V02          12345 non-null  int64  
 15  V03          12345 non-null  int64  
 16  V08          12345 non-null  int64  
 17  V27          12345 non-null  int64  
 18  VD_EDUCACAO  12345 non-null  int64  
 19  VD_R

### **4.2. Lidando com dados faltantes**

In [185]:
df.isna().sum()

P04               0
P50            3444
P51            3444
P52            3444
P54            3444
P55               0
P56               0
P57               0
P58               0
P59               0
P60               0
P61               0
P62               0
P63               0
V02               0
V03               0
V08               0
V27               0
VD_EDUCACAO       0
VD_REGIAO         0
VD_IDADE          0
P64               0
dtype: int64

In [186]:
df[df[['P50', 'P51', 'P52', 'P54']].isna().all(axis=1)].shape[0]

3444

In [187]:
df.isna().sum() / df.shape[0]

P04            0.000000
P50            0.278979
P51            0.278979
P52            0.278979
P54            0.278979
P55            0.000000
P56            0.000000
P57            0.000000
P58            0.000000
P59            0.000000
P60            0.000000
P61            0.000000
P62            0.000000
P63            0.000000
V02            0.000000
V03            0.000000
V08            0.000000
V27            0.000000
VD_EDUCACAO    0.000000
VD_REGIAO      0.000000
VD_IDADE       0.000000
P64            0.000000
dtype: float64

Como é possível observar, **há 3.444 valores nulos** para as variáveis **P50**, **P51**, **P52** e **P54**, que sempre ocorrem simultaneamente, ou seja, sempre que uma dessas variáveis é nula, todas as outras também o são. Essas ausências, entretanto, não representam dados faltantes no sentido tradicional, mas sim um caso de **ausência por desenho** do questionário (***missing by design***), que refere-se à prática intencional de **omitir dados** durante a coleta para otimizar a pesquisa ou para obter informações específicas, em vez de considerar os dados como perdidos por acidente ou erro. Conforme indicado no questionário, as perguntas P47 a P49 possuem uma lógica de salto: dependendo das respostas dadas pelo entrevistado, o entrevistador **avançará diretamente** para a pergunta P55.

<p style="text-align:center;">
  <img src="images/missing.png" alt="missing" style="max-width:100%; height:auto;">
</p>

Isso representa 3.444 linhas, quase **30% do conjunto de dados**. Assim, para não perder essa quantidade significativa de dados, será adotada a abordagem de substituir os valores faltantes por "*-1 : Não se aplica*".

In [188]:
cols = ['P50', 'P51', 'P52', 'P54']
mask = df[cols].isna().all(axis=1)
df.loc[mask, cols] = -1

df.isna().sum()

P04            0
P50            0
P51            0
P52            0
P54            0
P55            0
P56            0
P57            0
P58            0
P59            0
P60            0
P61            0
P62            0
P63            0
V02            0
V03            0
V08            0
V27            0
VD_EDUCACAO    0
VD_REGIAO      0
VD_IDADE       0
P64            0
dtype: int64

In [189]:
df.shape

(12345, 22)

### **4.3. Lidando com dados duplicados**

In [190]:
df.duplicated().sum()

2

In [191]:
df[df.duplicated(keep=False)]

,P04,P50,P51,P52,P54,P55,P56,P57,P58,P59,P60,P61,P62,P63,V02,V03,V08,V27,VD_EDUCACAO,VD_REGIAO,VD_IDADE,P64
1596,16,2.0,2.0,1.0,1.0,1,2,1,2,1,1,2,3,1,2,3,2,2,4,1,2,2
3754,53,4.0,1.0,1.0,1.0,1,2,1,1,2,2,1,1,1,2,3,1,2,4,5,3,1
3833,16,2.0,2.0,1.0,1.0,1,2,1,2,1,1,2,3,1,2,3,2,2,4,1,2,2
3980,53,4.0,1.0,1.0,1.0,1,2,1,1,2,2,1,1,1,2,3,1,2,4,5,3,1


Conforme os resultados acima, pode-se observar a presença de **dois dados duplicados**. No entanto, **eles não serão removidos**, haja vista que esse conjunto de dados se trata de uma pesquisa realizada com várias pessoas nos diversos estados do país, e que é pouco provável, mas não impossível, que duas pessoas do mesmo estado tenham exatamente as mesmas respostas para as mesmas perguntas.

### **4.4. Salvando os dados tratados**

O conjunto de dados tratado até aqui (ainda na forma numérica) não será descartado, pois será utilizado posteriormente para construir o modelo preditivo.

In [192]:
df.to_csv(r'data\processed\data2024.csv', sep=';', index=False)

### **4.5. Convertendo para valores textuais**

Finalmente, pode-se realizar o mapeamento dos valores numéricos para seus respectivos valores textuais, considerando o dicionário de dados disponível.

In [193]:
def map_df_to_text(df_to_translate: pd.DataFrame, path_excel: str, columns_names: list[str]) -> pd.DataFrame:
    """
    Função para mapear os valores numéricos do DataFrame para seus equivalentes textuais,
    por meio de um dicionário de dados fornecido em um arquivo Excel.

    Parâmetros:
    - df_to_translate: pd.DataFrame
        DataFrame contendo os dados com valores numéricos a serem traduzidos.
    - path_excel: str
        Caminho para o arquivo Excel que contém o dicionário de dados.
    - columns_names: list[str]
        Lista com os novos nomes das colunas para o DataFrame traduzido.
    """
    try:
        # --- Passo 1: Carregar o Dicionário de Dados (arquivo XLSX) ---
        df_excel = pd.read_excel(path_excel)

        # --- Passo 2: Dicionário para VALORES das colunas ---
        # Preenche as células vazias na coluna 'Código da variável' com o valor não nulo mais recente
        df_excel['Código da variável'] = df_excel['Código da variável'].ffill()

        # Máscara para filtrar apenas as linhas que contêm códigos válidos das categorias (não textuais)
        mask = pd.to_numeric(df_excel['Código da categoria'], errors='coerce').notna()

        # Filtra pela máscara e pelas colunas "Código da variável", "Código da categoria" e "Descrição da categoria"
        df_excel = df_excel.loc[mask, ['Código da variável', 'Código da categoria', 'Descrição da categoria']]
        df_excel['Código da categoria'] = pd.to_numeric(df_excel['Código da categoria'], errors='coerce').astype('Int64')

        # Cria o dicionário aninhado para a substituição de valores das colunas
        map_values = {}
        for var_code, group in df_excel.groupby('Código da variável'):
            map_values[str(var_code).strip()] = group.set_index('Código da categoria')['Descrição da categoria'].to_dict()

    except FileNotFoundError:
        print(f"Erro: O arquivo '{path_excel}' não foi encontrado.")
        exit()
    
    df_to_return = df_to_translate.replace(map_values)
    df_to_return.columns = columns_names

    return df_to_return

In [194]:
path_excel = r"data\raw\panorama_politico_06_2024\DATASEN BR 2024 06 BAROMETRO - DICIONARIO DE DADOS.xlsx"
columns_names = [ 'estado', 'motivo_compart_fnews', 'identificar_fnews', 'empresas_impedir_fnews', 'importancia_controle_fnews',
    'petrobras_combustiveis', 'uso_maconha', 'cotas_universidades', 'direito_aborto', 'pena_morte', 'posse_armas', 'confianca_urnas',
    'satisfacao_democracia', 'regime_governo', 'sexo', 'cor_raca', 'religiao', 'renda_familiar', 'escolaridade', 'regiao', 
    'faixa_etaria', 'posicionamento_politico']

df = map_df_to_text(df, path_excel, columns_names)

## **5. Visão geral dos dados tratados**

Logo abaixo está a relação entre os nomes das colunas e as perguntas feitas no questionário da pesquisa.

- **estado** - Em que estado você mora?
- **motivo_compart_fnews** - Em sua opinião, qual o principal motivo para as pessoas compartilharem uma notícia falsa nas redes sociais?
- **identificar_fnews** - Em sua opinião, é fácil ou difícil saber quais notícias são falsas nas redes sociais?
- **empresas_impedir_fnews** - Na sua opinião, as empresas donas das redes sociais deveriam ser responsáveis por impedir a divulgação de notícias falsas?
- **importancia_controle_fnews** - Para garantir uma disputa justa nas eleições, quão importante é o controle das notícias falsas nas redes sociais?
- **petrobras_combustiveis** - Você é a favor ou contra: "Diminuir o lucro da Petrobrás para reduzir o preço dos combustíveis"
- **uso_maconha** - Você é a favor ou contra: "Autorizar que as pessoas usem maconha como quiserem"
- **cotas_universidades** - Você concorda ou discorda: "O sistema de cotas para negros em universidades é justo."
- **direito_aborto** - Você concorda ou discorda: "As mulheres devem ter o direito de interromper a gravidez com segurança, caso elas queiram."
- **pena_morte** - Você concorda ou discorda: "Deveria existir pena de morte no Brasil."
- **posse_armas** - "Facilitar a posse de armas aumenta a segurança no Brasil."
- **confianca_urnas** - "O resultado das urnas eletrônicas em eleições é confiável."
- **satisfacao_democracia** - Em geral, qual o seu nível de satisfação com a democracia no Brasil?
- **regime_governo** - *(modificada para melhor entendimento)* Em sua opinião, qual o melhor regime de governo?
- **sexo** - Sexo
- **cor_raca** - Qual sua cor ou raça?
- **religiao** - Qual sua religião ou crença?
- **renda_familiar** - Qual foi a renda total da sua família no mês passado, somando as rendas e benefícios de todas as pessoas que moram com você, inclusive a sua?
- **escolaridade** - Escolaridade
- **regiao** - Região (Construída a partir da variável **estado**)
- **faixa_etaria** - Faixa etária
- **posicionamento_politico** - Na política se fala em esquerda, direita e centro. Você se considera mais de

In [195]:
df.shape

(12345, 22)

In [196]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12345 entries, 0 to 12344
Data columns (total 22 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   estado                      12345 non-null  object
 1   motivo_compart_fnews        12345 non-null  object
 2   identificar_fnews           12345 non-null  object
 3   empresas_impedir_fnews      12345 non-null  object
 4   importancia_controle_fnews  12345 non-null  object
 5   petrobras_combustiveis      12345 non-null  object
 6   uso_maconha                 12345 non-null  object
 7   cotas_universidades         12345 non-null  object
 8   direito_aborto              12345 non-null  object
 9   pena_morte                  12345 non-null  object
 10  posse_armas                 12345 non-null  object
 11  confianca_urnas             12345 non-null  object
 12  satisfacao_democracia       12345 non-null  object
 13  regime_governo              12345 non-null  ob

In [197]:
df.isna().sum()

estado                        0
motivo_compart_fnews          0
identificar_fnews             0
empresas_impedir_fnews        0
importancia_controle_fnews    0
petrobras_combustiveis        0
uso_maconha                   0
cotas_universidades           0
direito_aborto                0
pena_morte                    0
posse_armas                   0
confianca_urnas               0
satisfacao_democracia         0
regime_governo                0
sexo                          0
cor_raca                      0
religiao                      0
renda_familiar                0
escolaridade                  0
regiao                        0
faixa_etaria                  0
posicionamento_politico       0
dtype: int64

In [198]:
df.head()

,estado,motivo_compart_fnews,identificar_fnews,empresas_impedir_fnews,importancia_controle_fnews,petrobras_combustiveis,uso_maconha,cotas_universidades,direito_aborto,pena_morte,posse_armas,confianca_urnas,satisfacao_democracia,regime_governo,sexo,cor_raca,religiao,renda_familiar,escolaridade,regiao,faixa_etaria,posicionamento_politico
0,Paraná,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Contra,Contra,Discorda,Discorda,Discorda,Concorda,Discorda,Nada satisfeito(a),A democracia é sempre a melhor forma de governo.,Feminino,Branca,Católica,"Acima de R$ 8.472,00 (mais de 6 salários mínimos)",Ensino superior incompleto ou mais,Sul,De 40 a 49 anos,Direita
1,Sergipe,Não se aplica,Não se aplica,Não se aplica,Não se aplica,A favor,Contra,Discorda,Concorda,Concorda,Discorda,Discorda,Pouco satisfeito(a),Tanto faz ter um governo democrático ou govern...,Masculino,Parda,Católica,"Abaixo de R$ 2.824,00 (até 2 salários mínimos)",Até ensino fundamental incompleto,Nordeste,De 30 a 39 anos,Esquerda
2,Rio Grande do Norte,Confiam em quem mandou a notícia para elas,Difícil,Sim,Muito importante,A favor,Contra,Concorda,Concorda,Concorda,Concorda,Concorda,Muito satisfeito(a),"Em algumas situações, um governo autoritário é...",Feminino,Branca,Sem religião ou crença,"Acima de R$ 8.472,00 (mais de 6 salários mínimos)",Ensino médio completo,Nordeste,De 30 a 39 anos,Esquerda
3,Mato Grosso do Sul,Não sabem que a notícia é falsa,Difícil,Sim,Muito importante,A favor,Contra,Concorda,Discorda,Concorda,Concorda,Discorda,Nada satisfeito(a),A democracia é sempre a melhor forma de governo.,Feminino,Preta,Espírita,"Abaixo de R$ 2.824,00 (até 2 salários mínimos)",Ensino médio completo,Centro-Oeste,De 30 a 39 anos,Direita
4,Distrito Federal,Não se aplica,Não se aplica,Não se aplica,Não se aplica,A favor,Contra,Concorda,Concorda,Discorda,Discorda,Concorda,Pouco satisfeito(a),A democracia é sempre a melhor forma de governo.,Masculino,Preta,Evangélica,"Abaixo de R$ 2.824,00 (até 2 salários mínimos)",Ensino fundamental completo,Centro-Oeste,De 50 a 59 anos,Direita


In [199]:
df.describe()

,estado,motivo_compart_fnews,identificar_fnews,empresas_impedir_fnews,importancia_controle_fnews,petrobras_combustiveis,uso_maconha,cotas_universidades,direito_aborto,pena_morte,posse_armas,confianca_urnas,satisfacao_democracia,regime_governo,sexo,cor_raca,religiao,renda_familiar,escolaridade,regiao,faixa_etaria,posicionamento_politico
count,12345,12345,12345,12345,12345,12345,12345,12345,12345,12345,12345,12345,12345,12345,12345,12345,12345,12345,12345,12345,12345,12345
unique,27,7,4,4,5,3,3,3,3,3,3,3,4,4,2,6,6,4,4,5,5,3
top,Distrito Federal,Não se aplica,Fácil,Sim,Muito importante,A favor,Contra,Concorda,Discorda,Concorda,Discorda,Concorda,Pouco satisfeito(a),A democracia é sempre a melhor forma de governo.,Feminino,Parda,Católica,"Abaixo de R$ 2.824,00 (até 2 salários mínimos)",Ensino superior incompleto ou mais,Nordeste,De 40 a 49 anos,Direita
freq,508,3444,4525,6726,6957,9673,10191,6918,7160,6294,6647,6454,5528,9324,7501,5875,5488,5183,4869,4023,2867,6698


O tratamento dos dados foi concluído com sucesso. O conjunto final apresenta **12.345 respostas** válidas e **22 variáveis** relevantes, sem valores ausentes. A próxima etapa é a análise dos dados, onde serão investigadas as relações entre o posicionamento político e as demais características dos entrevistados.